<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Transactional Data Import Test </span> </p>

Uncomment the first two cells if you are debugging this test or running it separately.

In [0]:
//#!eval-notebook "../Initialization/InitSystemorphUtils"

In [0]:
//var pathToFiles = "../Files/";
//await InitializeDataNodesAsync(DataSource, pathToFiles, new string[] {"CH"});

In [0]:
var workspace = Workspace.CreateNew();
workspace.InitializeFrom(DataSource);

In [0]:
workspace.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>());

# Setting Constants

In [0]:
var EnableScenarioBackup = Scenarios.EnableScenario;

In [0]:
Scenarios.EnableScenario = true;

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);

In [0]:
var estimateTypes = (await workspace.Query<EstimateType>().ToArrayAsync());
var actualEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Actual)).Select(x => x.SystemName);
var cashflowEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Cashflow)).Select(x => x.SystemName);
var actualAndCashflowEstimateTypes = actualEstimateTypes.Intersect(cashflowEstimateTypes);
var onlyCashflowEstimateTypes = cashflowEstimateTypes.Except(actualEstimateTypes);

# CF-ACT Best Estimate

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,DAE,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,DAE,A,,-250
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(workspace).ExecuteAsync()

In [0]:
var ivsBestEstimateInit = await workspace.Query<IfrsVariable>().ToArrayAsync();

# CF Scenario

In [0]:
var cashflowsScenario = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,P,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-53.5,-52.5,-31.3
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
await Import.FromString(cashflowsScenario).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioInitCashFlows = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioInitCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(2);
cashflowVars.Length.Should().NotBe(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

In [0]:
//Scenario computes a LC instead of the CSM that is computed in BE
ivsScenarioInitCashFlows.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "C").Count().Should().NotBe(0);
ivsScenarioInitCashFlows.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "C").Sum(x => x.Values.First()).Should().Be(0);
ivsScenarioInitCashFlows.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "L").Count().Should().NotBe(0);
ivsScenarioInitCashFlows.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "L").Sum(x => x.Values.First()).Should().BeGreaterThan(0);

# ACT Scenario

In [0]:
var actualsScenario = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,NIC,A,,-308";

In [0]:
await Import.FromString(actualsScenario).WithFormat(ImportFormats.Actual).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioInitActuals = workspace.Query<IfrsVariable>().ToArray();
var diffs = ivsScenarioInitActuals.Except(ivsScenarioInitCashFlows, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(1);
cashflowVars.Should().BeEmpty();
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).Should().BeEmpty();

# CF Scenario : CU

Update cashflow for Scenario: 
1. undo the changes introduced with the previous scenario cashflow import => Actuals projections are reset.
2. scenario CU cashflow affects the Amortization Factor (and hence the Deferrables) is updated

In [0]:
var cashflowsScenarioUpdateCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,CU,P,CL,C,,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-7.5,-8.5";

In [0]:
await Import.FromString(cashflowsScenarioUpdateCU).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioCashflowUpdateCU = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioCashflowUpdateCU.Except(ivsScenarioInitActuals, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
//Actuals projections
ivsScenarioCashflowUpdateCU.Where(x => x.EstimateType==EstimateTypes.A).Count().Should().Be(1);
ivsScenarioCashflowUpdateCU.Single(x => x.EstimateType==EstimateTypes.A).AmountType.Should().Be(AmountTypes.NIC);

In [0]:
cashflowVars.Where(x => x.EstimateType == EstimateTypes.F).Count().Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.C).Count().Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.DA).Count().Should().NotBe(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# ACT Scenario : DAE

In [0]:
var actualsScenarioUpdateACAAEA = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,DAE,A,,-777";

In [0]:
await Import.FromString(actualsScenarioUpdateACAAEA).WithFormat(ImportFormats.Actual).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioActualUpdateDae = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioActualUpdateDae.Except(ivsScenarioCashflowUpdateCU, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
cashflowVars.Should().BeEmpty();
actualVars.Count().Should().Be(1);
diffs.Where(x => x.EstimateType == EstimateTypes.DA).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.F).Should().BeEmpty();
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).Should().BeEmpty();

# ACT Scenario : Remove DAE

Set DAE to zero and check that DA is changed

In [0]:
var actualsScenarioNoDae = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,DAE,A,,0";

In [0]:
await Import.FromString(actualsScenarioNoDae).WithFormat(ImportFormats.Actual).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioActualNoDaeEA = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioActualNoDaeEA.Except(ivsScenarioActualUpdateDae, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => x.EstimateType==EstimateTypes.A).Count().Should().Be(1);
diffs.Where(x => x.EstimateType == EstimateTypes.DA).ToArray().Length.Should().NotBe(0);
ivsScenarioActualNoDaeEA.Where(x => x.EstimateType == EstimateTypes.F).Count().Should().NotBe(0);
ivsScenarioActualNoDaeEA.Where(x => x.EstimateType == EstimateTypes.DA && x.AocType == AocTypes.EA && x.Values.Any(v => Math.Abs(v) > Precision)).Count().Should().Be(0);

# CF-ACT Scenario = BE

In [0]:
var cashflowsScenarioEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,DAE,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";
//BE CF
// DT1.1,PR,BE,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
// DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
// DT1.1,DAE,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
// DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
// DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
// DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
// DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
// DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
// DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
var actualsScenarioEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,DAE,A,,-250
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140"; 

In [0]:
await Import.FromString(actualsScenarioEqualToBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(workspace).ExecuteAsync()

In [0]:
await Import.FromString(cashflowsScenarioEqualToBestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportEqualToBestEstimate = await workspace.Query<IfrsVariable>().ToArrayAsync();

In [0]:
ivsScenarioReimportEqualToBestEstimate.Count().Should().Be(0);

# CF BestEstimate

Import of best estimate with different gross cash flow values, whereas reinsurance business has the same values as in the initialization. This causes an update of the best estimate IFRS variables but it must have no impact on Scenario variables because gross cash flow contains only a combination of AocType, Novelty and AmoutType that have already been imported for Scenario (see above the variable cashflowsScenarioEqualToBestEstimate). 

In [0]:
var cashflowsBestEstimateNoImpact = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,BE,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,DAE,BE,CL,C,,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26
DT1.1,CU,P,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";
//CF SCENARIO == BE
// DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
// DT1.1,PR,BE,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
// DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
// DT1.1,DAE,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
// DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
// DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";
// DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
// DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
// DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
// DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowsBestEstimateNoImpact).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlowsNoImpactScenario = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportCashFlowsNoImpactScenario.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
diffs.Where(x => x.EstimateType == EstimateTypes.DA).Should().NotBeEmpty();
actualVars.Length.Should().Be(3);
cashflowVars.Count().Should().NotBe(0);
actualAndCashflowVars.Count().Should().NotBe(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

In [0]:
//Gross has now LC 
diffs.Where(x => x.DataNode =="DT1.1" && x.EstimateType == "L").Count().Should().NotBe(0);
diffs.Where(x => x.DataNode =="DT1.1" && x.EstimateType == "C").Count().Should().Be(0);
//Reinsurance is only impacted in LoReCo as the BestEstimate LC has changed.
diffs.Where(x => x.DataNode =="DTR1.1" && x.EstimateType != "LR").Count().Should().Be(0);

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlowsNoImpactScenario = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlowsNoImpactScenario.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
//Scenario is back to have Gross in CSM (section 8) : LC variable with 0 value override the variables form BE
diffs.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "L" ).Count().Should().NotBe(0);
diffs.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "L" ).All(x => Math.Abs(x.Values.First()) < Precision).Should().BeTrue();
//Scenario CSM has no difference wrt original BE data
diffs.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "C" ).Count().Should().Be(0);

In [0]:
//Scenario Reinsurance has LoReCo variable with 0 valuses to override the variables form BE
diffs.Where(x => x.DataNode == "DTR1.1" && x.EstimateType == "LR" ).All(x => Math.Abs(x.Values.First()) < Precision).Should().BeTrue();
//Scenario Reinsurance has no other variable different from the initial BestEstimate
diffs.Where(x => x.DataNode == "DTR1.1" && x.EstimateType != "LR" ).Count().Should().Be(0);
//Which had Reinsurance variable in CSM
ivsBestEstimateInit.Where(x => x.DataNode == "DTR1.1"&& x.EstimateType == "C").Count().Should().NotBe(0);

In [0]:
//Updates of CF changes the projection of Written Actual producing BestEstimate variables != Scenario which are then created
ivsScenarioReimportCashFlowsNoImpactScenario.Where(x => x.EstimateType=="A").Count().Should().Be(3);

In [0]:
diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType) && 
                 x.EstimateType != EstimateTypes.F && x.Values.Any(v => Math.Abs(v) > Precision)).Count().Should().Be(0);
diffs.Where(x => x.EstimateType==EstimateTypes.A && x.Values.Any(v => Math.Abs(v) > Precision)).Count().Should().Be(0);

# CF BestEstimate - GIC -

Add ICO AmountType to the Gic cashflow.

In [0]:
var cashflowsBestEstimateImpact = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,ICO,BE,CL,C,,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3
DT1.1,PR,BE,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,BE,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,DAE,BE,CL,C,,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26
DT1.1,CU,P,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42";

In [0]:
await Import.FromString(cashflowsBestEstimateImpact).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlows = await workspace.Query<IfrsVariable>().ToArrayAsync();
var ivsBestEstimateReimportCashFlowsGic = ivsBestEstimateReimportCashFlows.Where(x => x.DataNode == "DT1.1");
var ivsBestEstimateReimportCashFlowsGric = ivsBestEstimateReimportCashFlows.Where(x => x.DataNode == "DTR1.1");
var diffs = ivsBestEstimateReimportCashFlows.Except(ivsBestEstimateReimportCashFlowsNoImpactScenario, IfrsVariableComparer.Instance());

In [0]:
//Projection of Written Actual for ICO is created in BE
ivsBestEstimateReimportCashFlows.Where(x => x.EstimateType=="A" && x.AmountType == "ICO").Count().Should().Be(1);

In [0]:
diffs.Count().Should().NotBe(0);
diffs.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "L").Should().NotBeEmpty();
diffs.Where(x => x.DataNode == "DTR1.1" && x.EstimateType == "LR").Should().NotBeEmpty();
diffs.Where(x => x.DataNode == "DTR1.1" && x.EstimateType != "LR").Should().BeEmpty();

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlows = await workspace.Query<IfrsVariable>().ToArrayAsync();

var ivsScenarioReimportCashFlowsGic = ivsScenarioReimportCashFlows.Where(x => x.DataNode == "DT1.1");
var ivsScenarioReimportCashFlowsGric = ivsScenarioReimportCashFlows.Where(x => x.DataNode == "DTR1.1");

var diffs = ivsScenarioReimportCashFlows.Except(ivsScenarioReimportCashFlowsNoImpactScenario, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
//For gross a new AT is carried over to Scenario, which affects the computed CSM 
diffs.Where(x => x.DataNode == "DT1.1" && x.EstimateType == "C").Should().NotBeEmpty();
diffs.Where(x => x.DataNode == "DT1.1" && x.EstimateType != "C").Should().BeEmpty();
//For Reinsurance No difference is expected
diffs.Where(x => x.DataNode == "DTR1.1").Should().BeEmpty();

In [0]:
//No variable for ICO is created in Scenario because the BE value is valid also in Scenario (Actuals - projection and BE)
ivsScenarioReimportCashFlows.Where(x => x.AmountType == "ICO").Should().BeEmpty();

In [0]:
cashflowVars.Should().BeEmpty();
actualVars.Length.Should().Be(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# CF BestEstimate - GIC Reimport-

Reimport of the same Cashflow file should not change the result for both imported Group of Contracts (GIC and GRIC).

In [0]:
await Import.FromString(cashflowsBestEstimateImpact).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlows2 = await workspace.Query<IfrsVariable>().ToArrayAsync();
var ivsBestEstimateReimportCashFlows2Gic = ivsBestEstimateReimportCashFlows2.Where(x => x.DataNode == "DT1.1");
var ivsBestEstimateReimportCashFlows2Gric = ivsBestEstimateReimportCashFlows2.Where(x => x.DataNode == "DTR1.1");

In [0]:
var gicDiffs = ivsBestEstimateReimportCashFlowsGic.Except(ivsBestEstimateReimportCashFlows2Gic, IfrsVariableComparer.Instance());
var gricDiffs = ivsBestEstimateReimportCashFlowsGric.Except(ivsBestEstimateReimportCashFlows2Gric, IfrsVariableComparer.Instance());

In [0]:
gicDiffs.Should().BeEmpty();
gricDiffs.Should().BeEmpty();

# ACT Best Estimate

In [0]:
var newActualsBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,NIC,A,,-2000
DT1.1,CF,PR,A,,4000
DT1.1,CF,CL,A,,-3000";

In [0]:
await Import.FromString(newActualsBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportActuals = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportActuals.Except(ivsBestEstimateReimportCashFlows, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
ivsBestEstimateReimportActuals.Where(x => x.EstimateType=="A")

In [0]:
diffs.Select(x => x.EstimateType).Distinct().All(x => new []{"C", "L", "A", "APA", "LR"}.Contains(x)).Should().BeTrue();

In [0]:
actualVars.Length.Should().NotBe(0);
cashflowVars.Length.Should().Be(0);

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);

var ivsScenarioReimportActuals = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportActuals.Except(ivsScenarioReimportCashFlows, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
//In section 8 ACT have been imported for Scenario => the import for BestEstimate actual does not affect Scenario actuals.
actualVars.Should().BeEmpty();

In [0]:
diffs.Where(x => x.EstimateType == EstimateTypes.C).Should().BeEmpty();
actualVars.Length.Should().Be(0);
cashflowVars.Length.Should().Be(0);

# CF Scenario

Reset Coverage Units for Scenario so that they are taken from Best Estimate. Then, test that a re-import of Best Estimate Coverage Units changes the Scenario Deferrable. 

In [0]:
var cashflowsScenarioNoCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-274.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var cashflowsBestEstimateUpdatedCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
DT1.1,NIC,BE,CL,C,,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33
DT1.1,DAE,BE,CL,C,,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26,-26
DT1.1,CU,P,CL,C,,-719,-71,-70,-70,-69,-69,-68,-68,-67,-67,-66,-66,-65,-65,-64,-64,-63,-63,-62,-62,-61,-61,-60,-60
DT1.1,,RA,CL,C,,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45";

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
await workspace.DeleteAsync(await workspace.Query<RawVariable>().ToArrayAsync());

In [0]:
await Import.FromString(cashflowsScenarioNoCU).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashflowNoCU = await workspace.Query<IfrsVariable>().ToArrayAsync();

In [0]:
await Import.FromString(cashflowsBestEstimateUpdatedCU).WithFormat(ImportFormats.Cashflow).WithTarget(workspace).ExecuteAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlowsWithCU = await workspace.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlowsWithCU.Except(ivsScenarioReimportCashflowNoCU, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => x.EstimateType==EstimateTypes.A).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
diffs.Count().Should().NotBe(0);

# Restore Constants

In [0]:
Scenarios.EnableScenario = EnableScenarioBackup;

In [0]:
workspace.Dispose();